In [ ]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

# CID : 카테고리(소설 : 1, 에세이 :55889 , 자기계발 : 336)

items = []
lists = []
results = []
KEY = '**********'

CID = 517

for page in range(100,494): # 페이지 입력
    print(page)
    
    url = f"https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={page}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID={CID}&SearchOption="
    
    res = requests.get(url) 
    res.raise_for_status() # 정상 200
    soup = BeautifulSoup(res.text, "lxml")
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        res = requests.get(link)
        soup = BeautifulSoup(res.text, "lxml")
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass
    
        # data = [ISBN]
        results.append(ISBN)

len(results)

In [ ]:
import pickle
ISBN336_01_51 = pd.DataFrame(results)
ISBN336_01_51.to_pickle('LINK_LIST_74_01_494.pkl')

In [12]:
# 링크
url_look_list = []

#API 키 발급
KEY = '********' 

isbn = ": 5000078190"

url_look_list.append('https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn))

# # 링크 저장
# for isbn in results:
#     url_look = 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn)
#     url_look_list.append(url_look)
# print(url_look_list[:5])
# print(len(url_look_list))

In [13]:
# 저장한 링크에서 책 정보 가져오기
import requests
import pandas as pd
import time

items = []
lists = []

for url_look in url_look_list:
    while True:
        try:
            response = requests.get(url_look)
            response.raise_for_status()  # 오류가 발생하면 예외를 발생

            text = response.json()  # JSON 응답을 파싱

            item = text.get('item', [])  # 'item' 키가 있는 경우 해당 값을 가져옴
            if item:
                # 도서 정보
                title = item[0].get('title', '')
                author = item[0].get('author', '')
                pubDate = item[0].get('pubDate', '')
                description = item[0].get('description', '')
                isbn13 = item[0].get('isbn13', '')
                categoryID = item[0].get('categoryId', '')
                categoryName = item[0].get('categoryName', '')
                salesPoint = item[0].get('salesPoint', '')
                adult = item[0].get('adult', '')
                rank = item[0].get('customerReviewRank', '')
                fullDes = item[0].get('fullDescription', '')
                fullDes2 = item[0].get('fullDescription2', '')
                story = item[0].get('subInfo', {}).get('story', '')

                # 리뷰 정보
                reviewList = item[0].get('reviewList', [])
                if reviewList:
                    reviewrank = reviewList[0].get('reviewRank', 0)
                    reviewtitle = reviewList[0].get('title', 0)
                    review_w = reviewList[0].get('writer', 0)
                else:
                    reviewrank = 0
                    reviewtitle = 0
                    review_w = 0

                # 리스트에 도서 정보를 추가
                lists.append([title, author, pubDate, description, isbn13, categoryID, categoryName,
                            salesPoint, adult, rank, fullDes, fullDes2, story, reviewrank, reviewtitle, review_w])
            else:
                print(text)
            time.sleep(0.5)
            break
        except requests.exceptions.RequestException as e:
            print(f"요청 중 오류 발생: {e}")

# 데이터프레임을 생성
cols = ['title', 'author', 'pubDate', 'description', 'isbn13', 'categoryID', 'categoryName',
        'salesPoint', 'adult', 'rank', 'fullDes', 'fullDes2', 'story', 'reviewrank', 'reviewtitle', 'review_w']
df = pd.DataFrame(lists, columns=cols)

import pickle
df.to_pickle('ISBN74_1_493.pkl', protocol=4)
### pkl 이름 ISBN[카테고리 번호]_[페이지 시작]_[페이지 끝] 으로 저장

{'errorCode': 10, 'errorMessage': '쿼리 제한횟수 일일 5천회를 초과하였습니다. 문의는 openapi@aladin.co.kr로 해주시기 바랍니다.'}


In [ ]:
df

In [17]:
import pandas as pd
df = pd.read_pickle('LINK_LIST_170_01_249.pkl')
filtered_df = df[df[0].str.contains(":")]
filtered_df
filtered_df.tail(10)

,0
33606,: 5000078190
33608,: 5000160051
33609,: 5000008915
33611,: 5000090010
33612,: 5000021411
33616,: 5000088849
33617,: 5000140985
33619,: 5000018337
33621,: 5000051564
33623,: 5000159260


In [1]:
import pandas as pd
df = pd.read_pickle('LINK_LIST_170_01_249.pkl')
df

,0
0,9791198517425
1,9791193869024
2,9791191056372
3,9791166571459
4,9791170610359
...,...
33620,9788971270035
33621,: 5000051564
33622,9788910302551
33623,: 5000159260
